In [ ]:
from labda.contexts import Contexts
from labda import Subject
from labda.spatial.locations import Location, Locations, TEMPLATES
from typing import Any
import numpy as np
from pathlib import Path
import geopandas as gpd
import pandas as pd


def check_locations(sbj: Subject, contexts: Contexts) -> dict[str, Any]:
    crs = sbj.metadata.crs
    df = sbj.df

    wear, days = sbj.get_wear_times(duration="10h")
    weekdays = days.iloc[0, -1]

    if weekdays < 3:
        raise ValueError("Not enough days of wear")

    home = Location.from_context(contexts.get_context("home"))
    school = Location.from_context(contexts.get_context("school"))
    contexts = {
        "home": home,
        "school": school,
    }
    contexts_obj = Locations(values=contexts.values())
    contexts_gdf = contexts_obj.gdf.set_crs(crs, allow_override=True).to_crs(
        "EPSG:4326"
    )
    contexts_gdf["osm"] = False

    school, school_prob = Locations.find_location(df, crs, **TEMPLATES["school"])
    home, home_prob = Locations.find_location(df, crs, **TEMPLATES["home"])

    home = home.values[0]
    home_buffer = home._get_geometry_in_crs(crs).centroid.buffer(100)

    school = school.values[0]
    school_buffer = school._get_geometry_in_crs(crs).centroid.buffer(100)

    buffers = {
        "home": Location(name="home_buffer", geometry=home_buffer),
        "school": Location(name="school_buffer", geometry=school_buffer),
    }

    buffers_obj = Locations(values=buffers.values())
    buffers_gpd = buffers_obj.gdf.set_crs(crs, allow_override=True).to_crs("EPSG:4326")
    buffers_gpd["osm"] = True
    buffers_gpd["buffer"] = True

    osm_locations = {
        "home": home,
        "school": school,
    }
    osm_locations_obj = Locations(values=osm_locations.values())
    osm_locations_gdf = osm_locations_obj.gdf
    osm_locations_gdf["osm"] = True
    osm_locations_gdf["buffer"] = False

    gdf = pd.concat([osm_locations_gdf, buffers_gpd, contexts_gdf], ignore_index=True)

    home = Location.get_polygon_intersection(
        buffers["home"].geometry,
        contexts["home"].geometry,
    )
    home["location"] = "home"
    home["id"] = sbj.id

    school = Location.get_polygon_intersection(
        buffers["school"].geometry,
        contexts["school"].geometry,
    )
    school["location"] = "school"
    school["id"] = sbj.id

    gdf[["name", "geometry", "amenity", "building", "osm", "buffer"]].to_file(
        f"{sbj.id}.geojson", driver="GeoJSON"
    )

    return [home, school]


project = list(Path("../../temp/IPEN_Denmark_Adolescents/Merged").glob("*.parquet"))

results = []

for file in project:
    # file = Path("../../temp/IPEN_Denmark_Adolescents/Merged/280102.parquet")
    sbj = Subject.from_parquet(file)
    contexts = Contexts.from_parquet(
        file.parent.parent / "Shapes" / f"{sbj.id}.parquet",
    )
    contexts.validate()

    try:
        result = check_locations(sbj, contexts)
        print(f"Subject: {sbj.id}")
        print(f"Home: {result[0]["intersection"]} ({round(result[0]["ratio"], 2)})")
        print(f"School: {result[1]["intersection"]} ({round(result[1]["ratio"], 2)})")
        print("---------------------")
        results += result
    except Exception as e:
        print(f"Error for subject {sbj.id}: {e}")
        print("---------------------")
        continue

Error for subject 160302: DataFrame must contain valid geometry data.
Error for subject 160301: DataFrame must contain valid geometry data.
Error for subject 260202: DataFrame must contain valid geometry data.
Error for subject 160209: DataFrame must contain valid geometry data.
Error for subject 560201: 3 validation errors for Locations
Error for subject 580101: 'NoneType' object has no attribute 'get'
Error for subject 590104: DataFrame must contain valid geometry data.

670301
280102

In [ ]:
import pandas as pd

analysis = pd.DataFrame(results)
analysis.loc[analysis["location"] == "home", "intersection"].value_counts()
analysis.to_csv("analysis.csv", index=False)

In [ ]:
from labda.spatial.locations import Locations, TEMPLATES
from PIL import ImageColor
import pydeck as pdk


home.get_polygon_intersection(home, context_home["geometry"])
school.get_polygon_intersection(school, context_school["geometry"])

locations = locations.gdf
locations.set_crs(crs, inplace=True, allow_override=True)
# locations = locations.to_crs(crs)
# locations["geometry"] = locations["geometry"].buffer(30)
locations.to_crs("EPSG:4326", inplace=True)
locations["color"] = locations["osm_id"].map(
    {
        589034813: ImageColor.getrgb("turquoise"),
        29383689: ImageColor.getrgb("orange"),
    }
)

layer_locations = pdk.Layer(
    "GeoJsonLayer",
    locations,
    stroked=True,
    filled=False,
    get_fill_color="color",
    get_line_color="color",
    get_radius=10,
    get_line_width=5,
    pickable=True,
)

In [ ]:
map_1.config

In [ ]:
from typing import Any

import pandas as pd
import pydeck as pdk
from pyproj import CRS

from labda.spatial.utils import df_to_gdf
from PIL import ImageColor
import pandas as pd


# df = temporal
# df["color"] = pd.NA
# df["color"] = df["color"].apply(lambda x: ImageColor.getrgb("white"))
# df["tooltip"] = df.apply(
#     lambda x: f"Datetime: {x.name}",
#     axis=1,
# )

gdf = contexts.gdf.copy()
gdf = gdf.loc[gdf["context"].isin(["home", "school"])]
gdf.to_crs("EPSG:4326", inplace=True)
gdf["color"] = gdf["context"].map(
    {
        "home": ImageColor.getrgb("green"),
        "school": ImageColor.getrgb("red"),
    }
)

layer = pdk.Layer(
    "GeoJsonLayer",
    gdf,
    stroked=True,
    filled=False,
    get_fill_color="color",
    get_line_color="color",
    get_radius=10,
    get_line_width=5,
    pickable=True,
)
layers = [layer, layer_locations]

center = gdf["geometry"].union_all().centroid
view_state = pdk.ViewState(longitude=center.x, latitude=center.y, zoom=15)

map = pdk.Deck(
    layers=layers,
    initial_view_state=view_state,
    tooltip={"html": "{tooltip}"},  # type: ignore
    # map_style=pdk.map_styles.CARTO_LIGHT,
)
map